In [1]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
%matplotlib inline

In [2]:
import seaborn as sns
import numpy as np

df = sns.load_dataset("iris")

In [3]:
def max_devided_min(x):
    return max(x) / min(x)

def max_minus_min(x):
    return max(x) - min(x)

aggs = {'nunique', 'count', 'sum', 'min', 'mean', 'median', 'max', 'var', 'quantile',  max_minus_min, max_devided_min}

In [4]:
X = df.drop(['sepal_length'], axis=1)
y = df['sepal_length']

In [5]:
from sklearn.model_selection import train_test_split
(X_train, X_test,
 y_train, y_test) = train_test_split(
    X, y, test_size=0.2, random_state=0,
)

In [6]:
X_train.head()

,sepal_width,petal_length,petal_width,species
137,3.1,5.5,1.8,virginica
84,3.0,4.5,1.5,versicolor
27,3.5,1.5,0.2,setosa
127,3.0,4.9,1.8,virginica
132,2.8,5.6,2.2,virginica


In [7]:
len(X_train)

120

In [8]:
from sklearn.model_selection import KFold

def target_encoding(X_train):

    kf = KFold(n_splits=5, shuffle=True, random_state=0)
    dfs = []

    for train_index, test_index in kf.split(X_train):
        X_used, X_unused = X.loc[train_index, :], X.loc[test_index, :]

        agg = X_used.groupby('species').agg(aggs)
        names_1 = agg.columns.get_level_values(0)
        names_2 = agg.columns.get_level_values(1)
        agg.columns = ['agg_' + el1 + str(el2) for (el1, el2) in zip(names_1, names_2)]
        tmp = pd.merge(X_unused, agg, on='species', how='left')
        tmp.index = test_index
        dfs.append(tmp)

    return pd.concat(dfs).sort_index()

In [9]:
X_train = target_encoding(X_train)

In [10]:
X_train.head()

,sepal_width,petal_length,petal_width,species,agg_sepal_widthnunique,agg_sepal_widthmax_devided_min,agg_sepal_widthvar,agg_sepal_widthquantile,agg_sepal_widthcount,agg_sepal_widthmax_minus_min,...,agg_petal_widthmax_devided_min,agg_petal_widthvar,agg_petal_widthquantile,agg_petal_widthcount,agg_petal_widthmax_minus_min,agg_petal_widthmin,agg_petal_widthsum,agg_petal_widthmean,agg_petal_widthmedian,agg_petal_widthmax
0,3.5,1.4,0.2,setosa,14,1.448276,0.109775,3.4,37,1.3,...,6.0,0.013078,0.2,37,0.5,0.1,9.0,0.243243,0.2,0.6
1,3.0,1.4,0.2,setosa,14,1.448276,0.109775,3.4,37,1.3,...,6.0,0.013078,0.2,37,0.5,0.1,9.0,0.243243,0.2,0.6
2,3.2,1.3,0.2,setosa,14,1.913043,0.146230,3.4,38,2.1,...,5.0,0.010071,0.2,38,0.4,0.1,9.2,0.242105,0.2,0.5
3,3.1,1.5,0.2,setosa,16,1.913043,0.157154,3.4,43,2.1,...,6.0,0.011595,0.2,43,0.5,0.1,10.6,0.246512,0.2,0.6
4,3.6,1.4,0.2,setosa,14,1.448276,0.109775,3.4,37,1.3,...,6.0,0.013078,0.2,37,0.5,0.1,9.0,0.243243,0.2,0.6


In [11]:
len(X_train)

120

In [12]:
def target_encoding_test(X_train, X_test):
    agg = X_train.groupby('species').agg(aggs)
    names_1 = agg.columns.get_level_values(0)
    names_2 = agg.columns.get_level_values(1)
    agg.columns = ['agg_' + el1 + str(el2) for (el1, el2) in zip(names_1, names_2)]
    return pd.merge(X_test, agg, on='species', how='left')

In [13]:
X_test = target_encoding_test(X_train, X_test)

In [14]:
X_test.head()

,sepal_width,petal_length,petal_width,species,agg_sepal_widthnunique,agg_sepal_widthmax_devided_min,agg_sepal_widthvar,agg_sepal_widthquantile,agg_sepal_widthcount,agg_sepal_widthmax_minus_min,...,agg_agg_petal_widthmaxmax_devided_min,agg_agg_petal_widthmaxvar,agg_agg_petal_widthmaxquantile,agg_agg_petal_widthmaxcount,agg_agg_petal_widthmaxmax_minus_min,agg_agg_petal_widthmaxmin,agg_agg_petal_widthmaxsum,agg_agg_petal_widthmaxmean,agg_agg_petal_widthmaxmedian,agg_agg_petal_widthmaxmax
0,2.8,5.1,2.4,virginica,11,1.727273,0.149053,2.95,20,1.6,...,1.000000,0.000000,2.5,20,0.0,2.5,50.0,2.500,2.5,2.5
1,2.2,4.0,1.0,versicolor,14,1.700000,0.098469,2.80,50,1.4,...,1.058824,0.001506,1.8,50,0.1,1.7,89.1,1.782,1.8,1.8
2,4.2,1.4,0.2,setosa,16,1.913043,0.143690,3.40,50,2.1,...,1.200000,0.001861,0.6,50,0.1,0.5,28.8,0.576,0.6,0.6
3,2.9,6.3,1.8,virginica,11,1.727273,0.149053,2.95,20,1.6,...,1.000000,0.000000,2.5,20,0.0,2.5,50.0,2.500,2.5,2.5
4,3.4,1.5,0.2,setosa,16,1.913043,0.143690,3.40,50,2.1,...,1.200000,0.001861,0.6,50,0.1,0.5,28.8,0.576,0.6,0.6


In [15]:
len(X_test)

30

In [16]:
def get_skew_kurt_from_df(df, col_name_agg, by_hoge):
    gp = df.groupby(by_hoge)[col_name_agg].agg(['mean', 'std']).reset_index().rename(columns = {'mean' : 'mu', 'std' : 's'})
    merged = pd.merge(df, gp, how = 'left', on = by_hoge)
    merged['z'] = (merged[col_name_agg] - merged['mu'])/merged['s']
    merged['z_pow_3'] = merged['z'] ** 3
    merged['z_pow_4'] = merged['z'] ** 4
    gp_z_pow_sum = merged.groupby(by_hoge)[['z_pow_3', 'z_pow_4']].sum()
    gp = df.groupby(by_hoge).size().reset_index().rename(columns = {0 : 'n'})
    assert(gp.shape[0] == gp_z_pow_sum.shape[0])
    gp['z_pow_3_sum'] = gp_z_pow_sum['z_pow_3'].values
    gp['z_pow_4_sum'] = gp_z_pow_sum['z_pow_4'].values
    gp['skew'] = (gp['n']/((gp['n'] - 1) * (gp['n'] - 2))) * gp['z_pow_3_sum']
    gp['kurt'] = ((gp['n'] * (gp['n'] + 1))/((gp['n'] - 1) * (gp['n'] - 2) * (gp['n'] - 3))) * gp['z_pow_4_sum'] \
    - (3 * ((gp['n'] - 1) ** 2))/((gp['n'] - 2) * (gp['n'] - 3))
    gp = gp.drop(['n', 'z_pow_3_sum', 'z_pow_4_sum'], axis = 1)
    return gp

In [17]:
get_skew_kurt_from_df(df, 'sepal_length', 'species')

,species,skew,kurt
0,setosa,0.120087,-0.252689
1,versicolor,0.105378,-0.533010
2,virginica,0.118015,0.032904
